In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np
import random



S = np.matrix([[0,1,2],[0,0,1],[0,0,0]])

t = [60,60,245]

D = [210, 210, 858]

q = [[[4.5,4.5,4.5],[4.5,4.5,4.5],[4.5,4.5,4.5]],
     [[5.5,5.5,5.5],[5.5,5.5,5.5],[5.5,5.5,5.5]],
     [[6.5,6.5,6.5],[6.5,6.5,6.5],[6.5,6.5,6.5]]]

b = [[30,  75,    37.5],
     [15,  37.5,  18.25],
     [7.5, 18.75, 9.325]]

w = 3000  

C = [[50, 50, 50],
     [50, 50, 50],
     [50, 50, 50]] 

r = [[[5, 5, 5], [5, 5, 5], [5, 5, 5]],
     [[6, 6, 6], [6, 6, 6], [6, 6, 6]],
     [[7, 7, 7], [7, 7, 7], [7, 7, 7]]]

M = 60.0  
H = 0.0  
V = [0.05, 0.05, 0.05] 
L = 3000.0  


t_matrix= [1,
 [0.14, 0.69, 0.17],
 [[0.14, 0.69, 0.17], [0.14, 0.69, 0.17], [0.14, 0.69, 0.17]],
 [[0.14, 0.69, 0.17], [0.14, 0.69, 0.17], [0.14, 0.69, 0.17]]]

stages = 4
states = 3





In [2]:

class MarkovChain:
    def __init__(self, t_matrix, stages, states):
        self.t_matrix = t_matrix
        self.stages = list(range(1,stages+1))
        self.states = list(range(1, states +1))

    def nodes(self):
        nodes = [(1,1)]
        for i in self.stages[1:]:
            for j in self.states:
                nodes.append((i,j))
        return nodes
    
    def edges(self):
        edges = {}
        for j in self.states:
                edges[(1,1),(2,j)] = t_matrix[1][j-1]
        for i in self.stages[2:]:
            for j in self.states:
                for k in self.states:
                    edges[(i-1,j),(i,k)] = t_matrix[i-1][j-1][k-1]
        return edges


    def sample_path(self):
        all_edges = self.edges()
        path = [(1,1)]
        for i in self.stages[1:]: 
            path.append((i,random.choice(self.states)))
        return path


class Node:
    def __init__(self, node, stages, states):
        self.node_name = node
        self.states = list(range(1, states +1))
        
        if node[0] == 1:
            self.parents  = None
            self.children = [(2,s) for s in self.states]
        elif node[0] == 2:
            self.parents = [(1,1)]
            self.children = [(3,s) for s in self.states]
        elif node[0] > 2 &  node[0] < stages:
            self.parents = [(node[0]-1,s) for s in self.states]
            self.children = [(node[0]+1,s) for s in self.states]
        elif node[0] == stages:
            self.parents  = [(node[0]-1,s) for s in self.states]
            self.children = None


class Stage:
    def __init__(self, stages, states):
        self.stages = list(range(1,stages+1))
        self.states = list(range(1, states +1))
        self.nodes = {}
        
        for i in self.stages:
            if i == 1:
                self.nodes[i] = [(1,1)]
            else:
                self.nodes[i] = [(i,j) for j in self.states]
            
    

In [3]:
mc = MarkovChain(t_matrix, stages, states)

Nodes = {}

for i in mc.nodes():
    Nodes[i] = Node(i,stages,states)

Pij = mc.edges()

Stages = Stage(stages,states)


In [4]:
def subproblem_builder():
    nodes = mc.nodes()
    subproblems = {}
    for i in nodes:
            stage = i[0]
            state = i[1]
            print("subproblem", i, "stage", stage, "state", state)

            m = ConcreteModel()
            
            #State Variables
            m.acres = Var(within = NonNegativeReals, bounds = (0,M))
            m.acres_in = Var(within =NonNegativeReals, bounds = (0,M))
            m.bales = Var([1,2,3], within=NonNegativeReals)
            m.bales_in = Var([1,2,3], within=NonNegativeReals)
            
            m.acres_x = Param(initialize=0, mutable=True)
            m.bales_x = Param([1,2,3], initialize=0, mutable=True)
            
            
            #Local Variables
            
            m.cost_to_go = Var(within=NonNegativeReals, initialize=0)
            

            
            if stage == 1:
            
                m.obj  = Objective(expr = 0.00001*m.acres + m.cost_to_go, sense= minimize)
                m.con1 = Constraint(expr = m.acres <= m.acres_in)
            
                def con2(m,b):
                    return m.bales[b] == m.bales_in[b]
                m.con2 = Constraint([1,2,3], rule = con2)
            
                m.con_acresx = Constraint(expr = m.acres_in == m.acres_x)
            
                def con_balesx(m,b):
                    return m.bales_in[b] == m.bales_x[b]
                m.con_balesx = Constraint([1,2,3], rule = con_balesx )
            
                m.cuts = ConstraintList()
            
            else:

                m.buy = Var([1,2,3], within=NonNegativeReals)
                m.sell = Var([1,2,3], within=NonNegativeReals)
                m.eat = Var([1,2,3], within=NonNegativeReals)
                m.pen_p = Var([1,2,3], within=NonNegativeReals)
                m.pen_n = Var([1,2,3], within=NonNegativeReals)
            
            
                m.obj  = Objective(expr = 1000 * (sum(m.pen_p[c] for c in [1,2,3]) + sum(m.pen_n[c] for c in [1,2,3])) + C[stage-2][state-1] * m.acres_in +
                sum( V[stage-2] * m.bales_in[c] * t[stage-2] +\
                     r[c-1][stage-2][state-1] * m.buy[c] +\
                     S[c-1, stage-2] * m.eat[c] -\
                     q[c-1][stage-2][state-1] * m.sell[c] for c in [1,2,3]) + m.cost_to_go) 
            
                m.con3 = Constraint(expr = m.acres <= m.acres_in)
            
                m.con4 = Constraint(expr = sum(m.eat[c] for c in [1,2,3]) >= D[stage-2])
            
                def cutex(m,b):
                    return m.bales_in[b] + m.buy[b] - m.eat[b] - m.sell[b] + m.pen_p[b] - m.pen_n[b]
                m.cutex = Expression([1,2,3], rule = cutex)
            
                m.con5 = Constraint(expr = m.bales[stage-1] == m.cutex[stage-1] + m.acres_in * b[stage-2][state-1])
                                    
                def con6(m,b):
                    return m.bales[b] == m.cutex[b]
                m.con6 = Constraint({1,2,3} - {stage-1}, rule = con6)
            
                m.con7 = Constraint(expr = sum(m.bales[b] for b in [1,2,3]) <= w)
            
                def con8(m,b):
                    return m.sell[b] <= m.bales_in[b]
                m.con8 = Constraint({1,2,3}, rule = con8)
            
                m.con9 = Constraint(expr = sum(m.sell[b] for b in [1,2,3]) <= L)
            
                m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
                
                m.con_acresx = Constraint(expr = m.acres_in == m.acres_x)
            
                def con_balesx(m,b):
                    return m.bales_in[b] == m.bales_x[b]
                m.con_balesx = Constraint([1,2,3], rule = con_balesx )
            
                m.cuts = ConstraintList()

            subproblems[i] = m
    return subproblems

sb = subproblem_builder()
opt = SolverFactory('gurobi')


subproblem (1, 1) stage 1 state 1
subproblem (2, 1) stage 2 state 1
subproblem (2, 2) stage 2 state 2
subproblem (2, 3) stage 2 state 3
subproblem (3, 1) stage 3 state 1
subproblem (3, 2) stage 3 state 2
subproblem (3, 3) stage 3 state 3
subproblem (4, 1) stage 4 state 1
subproblem (4, 2) stage 4 state 2
subproblem (4, 3) stage 4 state 3


In [5]:
for _ in range(20):
    
    iteration_1_path = mc.sample_path()
    print(iteration_1_path) 
    
    fpi = {}
    
    print("forward pass")
    
    node= iteration_1_path[0]
    
    print("node", node)
    
    sb[node].acres_x     = M
    sb[node].bales_x[1] =  H
    sb[node].bales_x[2] =  0 
    sb[node].bales_x[3] =  0 
    
    results = opt.solve(sb[node])
    
    acres_out =  sb[node].acres.value
    bales_out1 = sb[node].bales[1].value
    bales_out2 = sb[node].bales[2].value
    bales_out3 = sb[node].bales[3].value
    
    
    fpi[('acres',1)]  = sb[node].acres.value
    fpi[('bales1',1)] = sb[node].bales[1].value
    fpi[('bales2',1)] = sb[node].bales[2].value
    fpi[('bales3',1)] = sb[node].bales[3].value
    
    print(fpi)

    
    node = iteration_1_path[1]
    
    print("node", node)
    
    
    sb[node].acres_x    = acres_out
    sb[node].bales_x[1] = bales_out1 
    sb[node].bales_x[2] = bales_out2 
    sb[node].bales_x[3] = bales_out3
                  
    
    results = opt.solve(sb[node])
    
    acres_out = sb[node].acres.value
    bales_out1 = sb[node].bales[1].value
    bales_out2 = sb[node].bales[2].value
    bales_out3 = sb[node].bales[3].value
    
    fpi[('acres',2)] = sb[node].acres.value
    fpi[('bales1',2)] = sb[node].bales[1].value
    fpi[('bales2',2)] = sb[node].bales[2].value
    fpi[('bales3',2)] = sb[node].bales[3].value
    
    print(fpi)
 
    
    node = iteration_1_path[2]
    
    print("node", node)
    
    
    sb[node].acres_x    = acres_out
    sb[node].bales_x[1] = bales_out1 
    sb[node].bales_x[2] = bales_out2 
    sb[node].bales_x[3] = bales_out3
                  
    
    results = opt.solve(sb[node])
    
    acres_out = sb[node].acres.value
    bales_out1 = sb[node].bales[1].value
    bales_out2 = sb[node].bales[2].value
    bales_out3 = sb[node].bales[3].value
    
    fpi[('acres',3)] = sb[node].acres.value
    fpi[('bales1',3)] = sb[node].bales[1].value
    fpi[('bales2',3)] = sb[node].bales[2].value
    fpi[('bales3',3)] = sb[node].bales[3].value
    
    print(fpi)

    
    node = iteration_1_path[3]
    
    print("node", node)
    
    
    sb[node].acres_x    = acres_out
    sb[node].bales_x[1] = bales_out1 
    sb[node].bales_x[2] = bales_out2 
    sb[node].bales_x[3] = bales_out3
                  
    
    results = opt.solve(sb[node])
    
    acres_out = sb[node].acres.value
    bales_out1 = sb[node].bales[1].value
    bales_out2 = sb[node].bales[2].value
    bales_out3 = sb[node].bales[3].value
    
    fpi[('acres',4)] = sb[node].acres.value
    fpi[('bales1',4)] = sb[node].bales[1].value
    fpi[('bales2',4)] = sb[node].bales[2].value
    fpi[('bales3',4)] = sb[node].bales[3].value
    
    
    print(fpi)
    
    #------------------- Backward Pass -------------------# 
    
    
    β = {}
    α = {}
    
    print("\nbackward pass")
    
    opt = SolverFactory('gurobi')
    
    V = {}
    dxdV = {}
    
    # -----------------stage 4 cuts -------------#
    
    for node in Stages.nodes[4]:
        
        sb[node].cost_to_go.fix(0)
        sb[node].acres_x    = fpi[('acres',3)]
        sb[node].bales_x[1] = fpi[('bales1',3)]
        sb[node].bales_x[2] = fpi[('bales2',3)]
        sb[node].bales_x[3] = fpi[('bales3',3)]
    
    
        results = opt.solve(sb[node])
        V[node] = value(sb[node].obj)
        dxdV[(node,'acres')] = sb[node].dual[sb[node].con_acresx]
        dxdV[(node,'bales1')] = sb[node].dual[sb[node].con_balesx[1]]
        dxdV[(node,'bales2')] = sb[node].dual[sb[node].con_balesx[2]]
        dxdV[(node,'bales3')] = sb[node].dual[sb[node].con_balesx[3]]
    
        for i in Nodes[node].parents:
        
            β[i,node,'acres'] = Pij[i,node]*dxdV[(node,'acres')]
            β[i,node,'bales1'] = Pij[i,node]*dxdV[(node,'bales1')]
            β[i,node,'bales2'] = Pij[i,node]*dxdV[(node,'bales2')]
            β[i,node,'bales3'] = Pij[i,node]*dxdV[(node,'bales3')]
    
                
            α[i,node] = Pij[i,node]*V[node]\
                     - β[i,node,'acres']*fpi[('acres',3)]\
                     - β[i,node,'bales1']*fpi[('bales1',3)]\
                     - β[i,node,'bales2']*fpi[('bales2',3)]\
                     - β[i,node,'bales3']*fpi[('bales3',3)]\

    
    for i in Stages.nodes[3]:
        sb[i].cuts.add(expr = sb[i].cost_to_go >= sum(α[i,j] for j in Stages.nodes[4])\
                            + sum(β[i,j,'acres']*sb[i].acres for j in Stages.nodes[4])\
                            + sum(β[i,j,'bales1']*sb[i].bales[1] for j in Stages.nodes[4])\
                            + sum(β[i,j,'bales2']*sb[i].bales[2] for j in Stages.nodes[4])\
                            + sum(β[i,j,'bales3']*sb[i].bales[3] for j in Stages.nodes[4]))    
    
    
    
    # -----------------stage 3 cuts -------------#
    
    for node in Stages.nodes[3]:
        
        sb[node].acres_x = fpi[('acres',2)]
        sb[node].bales_x[1] = fpi[('bales1',2)]
        sb[node].bales_x[2] = fpi[('bales2',2)]
        sb[node].bales_x[3] = fpi[('bales3',2)]
    
    
        results = opt.solve(sb[node])
        V[node] = value(sb[node].obj)
        dxdV[(node,'acres')] = sb[node].dual[sb[node].con_acresx]
        dxdV[(node,'bales1')] = sb[node].dual[sb[node].con_balesx[1]]
        dxdV[(node,'bales2')] = sb[node].dual[sb[node].con_balesx[2]]
        dxdV[(node,'bales3')] = sb[node].dual[sb[node].con_balesx[3]]
    
        for i in Nodes[node].parents:
        
            β[i,node,'acres'] = Pij[i,node]*dxdV[(node,'acres')]
            β[i,node,'bales1'] = Pij[i,node]*dxdV[(node,'bales1')]
            β[i,node,'bales2'] = Pij[i,node]*dxdV[(node,'bales2')]
            β[i,node,'bales3'] = Pij[i,node]*dxdV[(node,'bales3')]
    
                
            α[i,node] = Pij[i,node]*V[node]\
                     - β[i,node,'acres']*fpi[('acres',2)]\
                     - β[i,node,'bales1']*fpi[('bales1',2)]\
                     - β[i,node,'bales2']*fpi[('bales2',2)]\
                     - β[i,node,'bales3']*fpi[('bales3',2)]\
    
    
    # Adding the cuts
    
    for i in Stages.nodes[2]:
        sb[i].cuts.add(expr = sb[i].cost_to_go >= sum(α[i,j] for j in Stages.nodes[3])\
                            + sum(β[i,j,'acres']*sb[i].acres for j in Stages.nodes[3])\
                            + sum(β[i,j,'bales1']*sb[i].bales[1] for j in Stages.nodes[3])\
                            + sum(β[i,j,'bales2']*sb[i].bales[2] for j in Stages.nodes[3])\
                            + sum(β[i,j,'bales3']*sb[i].bales[3] for j in Stages.nodes[3]))  
    
    
        
    # -----------------stage 2 cuts -------------#
    
    
    
    for node in Stages.nodes[2]:
        
        sb[node].acres_x = fpi[('acres',1)]
        sb[node].bales_x[1] = fpi[('bales1',1)]
        sb[node].bales_x[2] = fpi[('bales2',1)]
        sb[node].bales_x[3] = fpi[('bales3',1)]
    
    
        results = opt.solve(sb[node])
        V[node] = value(sb[node].obj)
        dxdV[(node,'acres')] = sb[node].dual[sb[node].con_acresx]
        dxdV[(node,'bales1')] = sb[node].dual[sb[node].con_balesx[1]]
        dxdV[(node,'bales2')] = sb[node].dual[sb[node].con_balesx[2]]
        dxdV[(node,'bales3')] = sb[node].dual[sb[node].con_balesx[3]]
    
        for i in Nodes[node].parents:
        
            β[i,node,'acres']  = Pij[i,node]*dxdV[(node,'acres')]
            β[i,node,'bales1'] = Pij[i,node]*dxdV[(node,'bales1')]
            β[i,node,'bales2'] = Pij[i,node]*dxdV[(node,'bales2')]
            β[i,node,'bales3'] = Pij[i,node]*dxdV[(node,'bales3')]
    
                
            α[i,node] = Pij[i,node]*V[node]\
                     - β[i,node,'acres']*fpi[('acres',1)]\
                     - β[i,node,'bales1']*fpi[('bales1',1)]\
                     - β[i,node,'bales2']*fpi[('bales2',1)]\
                     - β[i,node,'bales3']*fpi[('bales3',1)]\
    
    
    # Adding the cuts
    
    for i in Stages.nodes[1]:
        sb[i].cuts.add(expr = sb[i].cost_to_go >= sum(α[i,j] for j in Stages.nodes[2])\
                            + sum(β[i,j,'acres']*sb[i].acres for j in Stages.nodes[2])\
                            + sum(β[i,j,'bales1']*sb[i].bales[1] for j in Stages.nodes[2])\
                            + sum(β[i,j,'bales2']*sb[i].bales[2] for j in Stages.nodes[2])\
                            + sum(β[i,j,'bales3']*sb[i].bales[3] for j in Stages.nodes[2]))  
    
    
    
    
    
    
    node
    
    results = opt.solve(sb[1,1])
    V = value(sb[1,1].obj)
    lower_bound = V
    print(lower_bound)

[(1, 1), (2, 1), (3, 2), (4, 1)]
forward pass
node (1, 1)
{('acres', 1): 0.0, ('bales1', 1): 0.0, ('bales2', 1): 0.0, ('bales3', 1): 0.0}
node (2, 1)
{('acres', 1): 0.0, ('bales1', 1): 0.0, ('bales2', 1): 0.0, ('bales3', 1): 0.0, ('acres', 2): 0.0, ('bales1', 2): 0.0, ('bales2', 2): 0.0, ('bales3', 2): 0.0}
node (3, 2)
{('acres', 1): 0.0, ('bales1', 1): 0.0, ('bales2', 1): 0.0, ('bales3', 1): 0.0, ('acres', 2): 0.0, ('bales1', 2): 0.0, ('bales2', 2): 0.0, ('bales3', 2): 0.0, ('acres', 3): 0.0, ('bales1', 3): 0.0, ('bales2', 3): 0.0, ('bales3', 3): 0.0}
node (4, 1)
{('acres', 1): 0.0, ('bales1', 1): 0.0, ('bales2', 1): 0.0, ('bales3', 1): 0.0, ('acres', 2): 0.0, ('bales1', 2): 0.0, ('bales2', 2): 0.0, ('bales3', 2): 0.0, ('acres', 3): 0.0, ('bales1', 3): 0.0, ('bales2', 3): 0.0, ('bales3', 3): 0.0, ('acres', 4): 0.0, ('bales1', 4): 0.0, ('bales2', 4): 0.0, ('bales3', 4): 0.0}

backward pass
{((4, 1), 'acres'): -2.5, ((4, 1), 'bales1'): 0.0, ((4, 1), 'bales2'): 0.0, ((4, 1), 'bales3'): 0

In [ ]:
fpi = {}

print("forward pass\n")

node= (1,1)

print("node", node)

sb[node].acres_x     = M
sb[node].bales_x[1] =  H
sb[node].bales_x[2] =  0 
sb[node].bales_x[3] =  0 

results = opt.solve(sb[node])

acres_out =  sb[node].acres.value
bales_out1 = sb[node].bales[1].value
bales_out2 = sb[node].bales[2].value
bales_out3 = sb[node].bales[3].value


fpi[('acres',1)]  = sb[node].acres.value
fpi[('bales1',1)] = sb[node].bales[1].value
fpi[('bales2',1)] = sb[node].bales[2].value
fpi[('bales3',1)] = sb[node].bales[3].value

print("acres_out:",  acres_out)
print("cost_to_go:", sb[node].cost_to_go.value)
print("objective:", value(sb[node].obj))
print("stage objective:", value(sb[node].obj) - sb[node].cost_to_go.value)



node = (2,2)

print("\nnode", node)


sb[node].acres_x    = acres_out
sb[node].bales_x[1] = bales_out1 
sb[node].bales_x[2] = bales_out2 
sb[node].bales_x[3] = bales_out3
              

results = opt.solve(sb[node])

acres_out = sb[node].acres.value
bales_out1 = sb[node].bales[1].value
bales_out2 = sb[node].bales[2].value
bales_out3 = sb[node].bales[3].value

fpi[('acres',2)] = sb[node].acres.value
fpi[('bales1',2)] = sb[node].bales[1].value
fpi[('bales2',2)] = sb[node].bales[2].value
fpi[('bales3',2)] = sb[node].bales[3].value

print("acres_out:",  acres_out)
print("cost_to_go (Bellman Term):", sb[node].cost_to_go.value)
print("objective:", value(sb[node].obj))
print("stage objective:", value(sb[node].obj) - sb[node].cost_to_go.value)



node = (3,3)

print("\nnode", node)


sb[node].acres_x    = acres_out
sb[node].bales_x[1] = bales_out1 
sb[node].bales_x[2] = bales_out2 
sb[node].bales_x[3] = bales_out3
              

results = opt.solve(sb[node])

acres_out = sb[node].acres.value
bales_out1 = sb[node].bales[1].value
bales_out2 = sb[node].bales[2].value
bales_out3 = sb[node].bales[3].value

fpi[('acres',2)] = sb[node].acres.value
fpi[('bales1',2)] = sb[node].bales[1].value
fpi[('bales2',2)] = sb[node].bales[2].value
fpi[('bales3',2)] = sb[node].bales[3].value

print("acres_out:",  acres_out)
print("cost_to_go (Bellman Term):", sb[node].cost_to_go.value)
print("objective:", value(sb[node].obj))
print("stage objective:", value(sb[node].obj) - sb[node].cost_to_go.value)


node = (4,2)

print("\nnode", node)


sb[node].acres_x    = acres_out
sb[node].bales_x[1] = bales_out1 
sb[node].bales_x[2] = bales_out2 
sb[node].bales_x[3] = bales_out3
              

results = opt.solve(sb[node])

acres_out = sb[node].acres.value
bales_out1 = sb[node].bales[1].value
bales_out2 = sb[node].bales[2].value
bales_out3 = sb[node].bales[3].value

fpi[('acres',2)] = sb[node].acres.value
fpi[('bales1',2)] = sb[node].bales[1].value
fpi[('bales2',2)] = sb[node].bales[2].value
fpi[('bales3',2)] = sb[node].bales[3].value

print("acres_out:",  acres_out)
print("cost_to_go (Bellman Term):", sb[node].cost_to_go.value)
print("objective:", value(sb[node].obj))
print("stage objective:", value(sb[node].obj) - sb[node].cost_to_go.value)






In [7]:
sb[1,1].pprint()

6 Set Declarations
    bales_in_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    bales_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    bales_x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    con2_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    con_balesx_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    cuts_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   20 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}